In [1]:
from collections import defaultdict,Counter
from random import shuffle
from sklearn.model_selection import train_test_split
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model

In [2]:
allReviews = []
with open('../CA_reviews.txt',encoding='utf-8') as f:
    for l in f:
        allReviews.append(eval(l))
            
middleSet,testSet = train_test_split(allReviews,test_size=0.2)
trainSet,validSet = train_test_split(middleSet,test_size=0.125)

In [3]:
def get_rating(data):
    rating_list = []
    for i in data:
        if i['reviewTime'] is None or i['gps'] is None:
            continue
        rating_list.append(i['rating'] if i['rating'] is not None else 0)
    return rating_list

# get_rating(validSet)

In [4]:
def get_feature(data):
    feature_list = []
    for i in data:
        if i['reviewTime'] is None or i['gps'] is None:
            continue
        temp = []
        temp.append(0 if i['reviewText'] == None else len(i['reviewText']))
        temp.append(i['gps'][0])
        temp.append(i['gps'][1])
        temp.append(int(i['reviewTime'].split(' ')[-1]))
#         temp.append(int(i['gPlusPlaceId']))
#         temp.append(int(i['gPlusUserId']))
        feature_list.append(temp)
    return feature_list
# get_feature(validSet)

In [5]:
def get_mse(ans, label):
    mse = 0
    for i,j in zip(ans, label):
        mse += (i - j) ** 2
    mse /= len(ans)
    return mse

In [6]:
X_train = get_feature(trainSet)
y_train = get_rating(trainSet)
X_valid = get_feature(validSet)
y_valid = get_rating(validSet)
X_test = get_feature(testSet)
y_test = get_rating(testSet)

In [7]:
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [8]:
model = xgb.XGBRegressor(max_depth=10, learning_rate=0.5, n_estimators=80, silent=False, objective='reg:squarederror')
model.fit(X_train, y_train)
ans = model.predict(X_test)
# score_ensemble = accuracy_score(ans, y_test)

In [9]:
best_model = None
best_mse = 10
def grid_search(depth, rate, estimators):
    temp_mse = best_mse
    model = xgb.XGBRegressor(max_depth=depth, learning_rate=rate, n_estimators=estimators, silent=False, objective='reg:squarederror')
    model.fit(X_train, y_train)
    ans = model.predict(X_valid)
    return model, ans
        
        
        
depth = [5,10,15]
rate = [0.1,0.3,0.6,0.9]
estimators=[40,80,160]

for i in depth:
    for j in rate:
        for k in estimators:
            model, ans = grid_search(i,j,k)
            if get_mse(ans, y_valid) < best_mse:
                best_model = model
                best_mse = get_mse(ans, y_valid)

In [10]:
pred = best_model.predict(X_test)
print('Best Mean Squared Error on validation set is: ' + str(best_mse))
print('Best Mean Squared Error on testing set is: ' + str(get_mse(pred, y_test)))

Best Mean Squared Error on validation set is: 1.3186185390174163
Best Mean Squared Error on testing set is: 1.3170175820944408
